## データセットの読み込み

In [ ]:
# type: ignore
import pandas as pd

train_org = pd.read_csv("data/train.csv", index_col="id")
train_ext = pd.read_csv("data/training_extra.csv", index_col="id")
train = pd.concat([train_org, train_ext], axis=0, ignore_index=False)
# test = pd.read_csv("data/test.csv")
# submission = pd.read_csv("data/sample_submission.csv")

## データ全体の確認

まずは、トレーニングデータセット内のデータ数や欠損値の有無、データ型をざっくりと確認します。
データ数は 30 万件で、多くの特徴量はカテゴリ変数であることがわかります。

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
train.describe().round(2)

## カテゴリデータについて調べる

### カテゴリ変数のユニークな値を確認

In [ ]:
categorical_columns = train.select_dtypes(include=["object"]).columns
for col in categorical_columns:
    print(f"{col} 列のユニーク値: {train[col].unique()}")

### One-hot エンコーディング

In [ ]:
# type: ignore
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# カテゴリ変数のカラム一覧を取得
categorical_columns = train.select_dtypes(include=["object"]).columns.to_list()

# 各カラムの変換器を作成（カテゴリ変数は One-hot エーコーディング、それ以外はそのまま残す）
col_transformer = ColumnTransformer(
    transformers=[("OneHot", OneHotEncoder(sparse_output=False), categorical_columns)],
    remainder="passthrough"
)

# 実際に変換してみる
col_transformer.fit(train)
train_onehot = pd.DataFrame(
    data=col_transformer.transform(train),
    columns=col_transformer.get_feature_names_out()
)
train_onehot